# Лабораторна робота №5 ІАД

<b>Виконала:</b> студентка МІТ-31, Панченко Владислава

<b>Завдання:</b>
1. Створити набір даних за допомогою DatasetGenerator.
2. Завантажити набір даних для класифікації з сайту Kaggle. Кожен студент має обрати унікальний набір даних.
3. (Опціонально для отримання відмінної оцінки) Згенерувати набір даних для класифікації, використовуючи стандартні можливості Python (наприклад, з допомогою бібліотек Numpy або random).
4. Для отриманих наборів даних провести встановленні кроки моделювання.
5. Викласти результати на GitHub у власний репозиторій в основну гілку в папці Lab5.

1. (Опціонально для отримання відмінної оцінки) Згенерувати набір даних для класифікації, використовуючи стандартні можливості Python (наприклад, з допомогою бібліотек Numpy або random).
2. Для отриманих наборів даних провести встановленні кроки моделювання.

In [14]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)

data = pd.DataFrame(X, columns=[f'f_{i+1}' for i in range(X.shape[1])])
data['target'] = y
print("Перші рядки даних:")
print(data.head())

data.iloc[0:10, 0] = np.nan
print("\nПеревірка пропусків:")
print(data.isnull().sum())

data.fillna(data.mean(), inplace=True)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data.drop('target', axis=1))
data['category'] = np.random.choice(['A', 'B', 'C'], size=data.shape[0])
encoder = OneHotEncoder(sparse_output=False)
category_encoded = encoder.fit_transform(data[['category']])
category_encoded_df = pd.DataFrame(category_encoded, columns=encoder.get_feature_names_out(['category']))
data = pd.concat([data, category_encoded_df], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, data['target'], test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("\nТочність моделі:", accuracy)


Перші рядки даних:
        f_1       f_2       f_3       f_4       f_5       f_6       f_7  \
0  0.964799 -0.066449  0.986768 -0.358079  0.997266  1.181890 -1.615679   
1 -0.916511 -0.566395 -1.008614  0.831617 -1.176962  1.820544  1.752375   
2 -0.109484 -0.432774 -0.457649  0.793818 -0.268646 -1.836360  1.239086   
3  1.750412  2.023606  1.688159  0.006800 -1.607661  0.184741 -2.619427   
4 -0.224726 -0.711303 -0.220778  0.117124  1.536061  0.597538  0.348645   

        f_8       f_9      f_10  target  
0 -1.210161 -0.628077  1.227274       0  
1 -0.984534  0.363896  0.209470       1  
2 -0.246383 -1.058145 -0.297376       1  
3 -0.357445 -1.473127 -0.190039       0  
4 -0.939156  0.175915  0.236224       1  

Перевірка пропусків:
f_1       10
f_2        0
f_3        0
f_4        0
f_5        0
f_6        0
f_7        0
f_8        0
f_9        0
f_10       0
target     0
dtype: int64

Точність моделі: 0.83


https://www.kaggle.com/datasets/anshulmehtakaggl/60k-responses-of-16-personalities-test-mbt
1. Завантажити набір даних для класифікації з сайту Kaggle. Кожен студент має обрати унікальний набір даних.
2. Для отриманих наборів даних провести встановленні кроки моделювання.


In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

df = pd.read_csv('16P.csv', encoding='windows-1252')

print("Інформація про завантажений набір даних:")
print(df.info())
print("\nПерші кілька рядків даних:")
print(df.head())

print("\nПеревірка пропусків:")
print(df.isnull().sum())
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)
print("\nПропуски після обробки:")
print(df.isnull().sum())

print("\nКатегоріальні змінні:")
print(categorical_columns)
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_categorical_data = encoder.fit_transform(df[categorical_columns])
encoded_df = pd.DataFrame(encoded_categorical_data, columns=encoder.get_feature_names_out(categorical_columns))
df = df.drop(categorical_columns, axis=1)
df = pd.concat([df, encoded_df], axis=1)

X = df.drop('Response Id', axis=1)
y = df['Response Id']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
if X_train.shape[1] > 100:
    X_train = X_train[:, :100]
    X_test = X_test[:, :100]

model = SGDClassifier(loss='log_loss', max_iter=1000, random_state=42, class_weight='balanced')
model.fit(X_train, y_train)
print("Розмірність X_train до оптимізації:", X_train.shape)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точність моделі:", accuracy)

Інформація про завантажений набір даних:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59999 entries, 0 to 59998
Data columns (total 62 columns):
 #   Column                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                --------------  ----- 
 0   Response Id                                                                                                           59999 non-null  int64 
 1   You regularly make new friends.                                                                                       59999 non-null  int64 
 2   You spend a lot of your free time exploring various random topics that pique your interest                            59999 non-null  int64 
 3   Seeing other people cry can easily make you feel like you want to cry too                                          

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9416\1909566467.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


KeyboardInterrupt: 